In [1]:
!pip install degiro_connector pandas scipy sqlitedict polars yfinance
!pip install selenium


In [2]:
from screener import Screener 
import logging
import sys
import pandas as pd
import csv
import numpy as np

pd.set_option("display.max_colwidth", 40)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

logfile = "./outdegiro.log"
logging_level = logging.INFO
logging.basicConfig(
    level=logging_level,
    handlers=[logging.FileHandler(logfile, mode="a"), logging.StreamHandler(sys.stdout), ],
    format="%(asctime)s - %(name)s:%(filename)s:%(funcName)s:%(lineno)d - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

'''
logging.getLogger("yfinance").setLevel(logging.DEBUG)
logging.getLogger("requests_cache").setLevel(logging.DEBUG)
logging.getLogger("cachedYahooApi").setLevel(logging.DEBUG)

main_logger = logging.getLogger()
'''

df = Screener(None, None, None, ["GB"])


2025-10-15 17:49:25 - degiro_connector.trading.actions.action_connect:action_connect.py:get_session_id:100 - CRITICAL - status: 200  application/json;charset=ISO-8859-1 {"captchaRequired":false,"isPassCodeEnabled":true,"locale":"fr_FR","redirectUrl":"https://trader.deg
2025-10-15 17:49:25 - degiro_connector.trading.actions.action_connect:action_connect.py:get_session_id:140 - INFO - login_sucess: {'isPassCodeEnabled': True, 'locale': 'fr_FR', 'redirectUrl': 'https://trader.degiro.nl/trader/', 'sessionId': 'A589E7A966B060330495470BE312D3CC.prod_dcv_ch63_3', 'status': 0, 'statusText': 'success'}
2025-10-15 17:49:28 - root:cachedDegiroApi.py:connect:60 - WARNING - You are now connected, with the session id:A589E*********ch63_3
2025-10-15 17:49:31 - root:screener.py:access1country:235 - WARNING - country:GB list:All (100 stocks for page 1)
2025-10-15 17:49:33 - root:screener.py:access1country:235 - WARNING - country:GB list:All (100 stocks for page 2)
2025-10-15 17:49:33 - root:screener.py

In [3]:

if df is not None:
    QS = 80.0  # score loic
    QSP = 95.0  # score loic + perf
    REV = 5   # croissance revenu
    MRG = 10   # marge
    SLD = .2  # en solde de x%
    LH = 1  # cours relatif entre le plus bas annuel et le plus haut [0,1]
    YLD = 17  # rendement dividende+prix
    PRX = 0   # croissance annuelle du prix de l'action, sans les dividendes
    ddf = df.copy()
    
    ddf = ddf[
        (ddf["qscore"] >= QS)
        & (ddf["qscorePerf"] >= QSP)
        & (ddf["REVPS5YGR"] >= REV)
        & (ddf["MARGIN5YR"] >= MRG)
        & (ddf["En Solde"] >= SLD)
        & (ddf["L%H"] <= LH)
        & (ddf["ChPctPrice5Y"] >= PRX)
        & (ddf["YLD+PRY"] >= YLD)
    
    ]
    
    ddf = ddf[
        [
            "YSymbol", "isin", "name", "sector", "industry", "country", "score", "qscore", "scorePerf", "qscorePerf", "MKTCAP.USD","qMKTCAP.USD", "MKTCAP","priceCurrency", "REVPS5YGR", "MARGIN5YR", "PER", "Rendement", "En Solde", "L%H", "ChPctPrice5Y", "AROE5YAVG", "closePriceDate",
        ]
    ]
    
    print(ddf.shape)
    display(ddf)
    
    #ddf.to_csv("extrait.csv", index=False, sep=str(";"), decimal=str(","), encoding="utf-8-sig", float_format="%.3f", quoting=csv.QUOTE_MINIMAL)


(5, 23)


,YSymbol,isin,name,sector,industry,country,score,qscore,scorePerf,qscorePerf,MKTCAP.USD,qMKTCAP.USD,MKTCAP,priceCurrency,REVPS5YGR,MARGIN5YR,PER,Rendement,En Solde,L%H,ChPctPrice5Y,AROE5YAVG,closePriceDate
3,NaN,ZAE000296554,THUNGELA RESOURCES LTD,Energy,Coal (NEC),South Africa (ZAF),"8,379.12",100,131.07,99,"688,478,527.34",66,"11,925,010,000.00",ZAR,13.91,22.65,4.42,15.32,0.41,0.04,22.53,37.28,2025-10-14
154,NaN,IL0011284465,PLUS500 LTD,Technology,Miscellaneous Fintech Infrastructure,Israel (ISR),285.69,81,133.07,100,"2,997,064,522.40",82,"2,244,824,000.00",GBP,25.21,44.04,11.94,4.67,0.28,0.81,16.55,54.58,2025-10-14
433,NaN,GB00BLDRH360,OSB GROUP PLC,Financial,Retail & Mortgage Banks,United Kingdom (GBR),"1,016.68",91,129.93,99,"2,648,934,527.20",81,"1,984,072,000.00",GBP,8.42,47.03,7.77,5.96,0.39,0.90,13.01,15.48,2025-10-14
743,NaN,GB00B0XLRJ79,B.P. MARSH & PARTNERS PLC,Financial,Investment Management & Fund Operato...,United Kingdom (GBR),"1,522.92",95,138.59,100,"331,251,126.96",56,"248,109,600.00",GBP,44.80,82.33,2.68,1.58,0.27,0.69,22.47,20.38,2025-10-14
1064,NaN,GB0000365774,ANGLO-EASTERN PLANTATIONS PLC,Consumer/Non-Cyclical,Fishing & Farming (NEC),United Kingdom (GBR),"2,181.58",97,130.97,99,"711,807,496.92",67,"533,149,200.00",GBP,11.25,20.72,8.15,2.54,0.27,0.96,23.19,15.11,2025-10-14
